In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
from scipy.stats import norm
from BlackScholes import (BS_call, BS_bull_call,BS_bull_call_delta)
from MC_bull import (MC_bull_call_naive, MC_bull_call_ant, MC_bull_call_con, MC_bull_call_imp, MC_bull_call_delta_ant_path)
from mpl_toolkits.mplot3d import Axes3D  
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

# Pricing the Bull Call Spread using Monte Carlo Methods

In the following project we will price and investigate a type of option contract known as a bull call spread using the black-scholes model and various monte carlo methods.

In the following project we will simulate 1000 stock price paths for a stock price ranging from £10 to £200 in steps of £5, while assuming constant annual interest rates, $r$ = 3%, and annual volatility, $\sigma$ = 25%. Using these stock price paths we will price a bull call spread with strike prices $K_{1}$ = £90 and $K_{2}$ = £120, with a time to maturity $T$ of 18 months. Wehere the underlying asset $S(T)$ follows geometric Brownian Motion:

$$ dS(t) = rS_tdt + \sigma S_{t}dW_{t} $$

Where the payoff $f(S(T))$ of a bull call spread for a certain stock price $S(T)$ at time $T$ defined as:

$$f(S(T)) = max(S(T) - K_{1},0) - max(S(T) - K_{2},0)$$

The Bull Call Spread will be priced initially using monte carlo methods without any variance reduction, refered to as the "naive" method throughout the project, and will then be priced again using three different variance reduction techniques:
1. Antithetic variance reduction
2. Use of control variates
3. Use of importance sampling

From the variances of these simulations, we will deterimine the sample size needed such that the absolute accuracy of the Monte Carlo price is £0.01, with a 95% confidence level, over the range of asset prices.

We will also price and plot the delta of this option.

In [ ]:
# Setting parameters to be used in bull call spread calculation
#S = 110

# N = Number of simulations per stock price S
N = 1000

# Strike Price for the call we buy = K1 = £90, strike price for the call we sell = K2 = £120, 
K1 = 90
K2 = 120

#Time measured in years hence 18mths = 1.5yrs
T = 1.5 

#Interest rate = 3%
r = 0.03

#Volatility = 25%
sigma = 0.25

For illustration purposes and to be used as a reference, the following is a plot of the bull call spread solved numerically using the black scholes model before we start our monte carlo analysis.

In [ ]:
#BlackScholes Plot
#setting list of stock prices we will calculate the spread value for

S_plot_BS = np.arange(10,205,5)
Npts = len(S_plot_BS)
BSprice = np.zeros(Npts)
#loop to calculate the price using BS Bull Call function in our python module
for k in range(Npts):
    BSprice[k] = BS_bull_call(S_plot_BS[k],K1,K2,T,r,sigma)
    
#Plotting    
plt.plot(S_plot_BS, BSprice,marker = 'o', linestyle = '',color = 'r')
plt.xlabel("Spot price S", fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Bull Call Spread Price for various Spot Prices Using Black Scholes")
plt.show()

# Naive Monte Carlo (No Variance Reduction)

### Naive Monte Carlo 

In [ ]:
#Running naive montecarlo for asset prices from S = 10 to S = 200 in steps of 5 for an inital 1000 simulations.
# N = number of simulations

# S_plot = array of stock prices ranging from 10 to 200 inclusive in steps of 5
S_plot = np.arange(10,205,5)

#Npts is equal to the number of stock prices we will calculate the price of the bull call spread for
Npts = len(S_plot)

#Setting empty arrays for which the monte carlo bull call prices variances will be stored in for corresponding stock prices
MC_price_naive = np.zeros(Npts)
MC_var_naive = np.zeros(Npts)

# Running 1000 simulations for each stock price in our S_plot array and calculating the price and variance of each.
for k in range(Npts):
    MC_price_naive[k], MC_var_naive[k] = MC_bull_call_naive(S_plot[k],K1,K2,T,r,sigma,N)
    
#For illustation purposes we will plot the bull call spread prices with 1000sims against S before doing anything else.

plt.plot(S_plot,MC_price_naive , label = "Naive")

plt.xlabel("Spot price S")
plt.ylabel("Bull Call Spread Price")
plt.title("Plot of bull call spread price for naive MC method ")
plt.legend()
plt.show()

#Saving variances for later plots
var_naive = MC_var_naive

In order for the accuracy of the simulated prices to have a maximum absolute error of £0.01, we require N simulations, where N is determined by

$$ ||V_{MC} - V||_{\infty} = 0.01 \geq 1.96\frac{\sigma_{M}}{\sqrt{N}} $$

$$ N \geq \left(\frac{1.96\sigma_{M}}{0.01}\right)^2$$

where we define $\sigma_{M}$ as the maximum variance of the option price after a preliminary run of 1000 simulations. 

We use the max variance in this calculation so that we can achieve the required accuracy for every stock price ranging from £10 to £200 in steps of 5, by changing the number of simulations from our initial value of 1000 to our value of N defined above.

To compare the four computation methods of our barrier option we will do this and determine a specific N for the naive MC method and for the three variance reduction techniques that will ensure absolute error or £0.01 across all methods.

In [ ]:
#Plotting Stock price vs variance for each MC calculation of the bull call spread.
plt.plot(S_plot,MC_var_naive, label = "Naive")

plt.xlabel("Spot price S")
plt.ylabel("Variance")
plt.title("Variance plot of bull call spread for naive MC method ")
plt.legend()
plt.show()
    

In [ ]:
# Use max variance to determine sample sizes according to our definition above
# Setting the error to our required £0.01
Error = 0.01

#Max_var = SigmaM
Max_var = np.amax(MC_var_naive)
print("Max variance of naive MC is", '{0:.5g}'.format(Max_var), '\n')

# work out N needed for Error < 0.01
N = (1.96/Error)**2 * Max_var
print("For Error =", Error, "the needed sample size is", '{0:.2g}'.format(N),'\n')

#from our initial run the sample size is in the millions so it would make sense to see another decimal place of accuracy for illustration purposes
print("For Error =", Error, "the needed sample size is", '{0:.3g}'.format(N))


#Saving Value for later
Nnaive = N

In [ ]:
# Testing for a single value of the stock price in the region where the max variance occurred as a check that we
# are after successfully reducing the standard error

# Test parameters
S = 109 #Note how roughly a stock price of 110 results in the maximum variance in the variance plot

#Our New sample size N as defined above (must be a whole number)
N = int((1.96/Error)**2 * Max_var)

MC_price_naive, MC_var_naive = MC_bull_call_naive(S,K1,K2,T,r,sigma,N)

SE = np.sqrt(MC_var_naive/N)

print("Naive MC price is ", '£{0:.5g}'.format(MC_price_naive), "+/-", '£{0:.2g}'.format(1.96*SE) )

print("Standard Error = ",'£{0:.2g}'.format(SE))



In [ ]:
#Now with our new sample size of N, we will price the spread using the naive monte carlo method (takes a few seconds due to using 6m sims)


S_plot = np.arange(10,205,5)

Npts = len(S_plot)
price_plot_naive = np.zeros( Npts )
var_plot_naive = np.zeros( Npts )


for k in range(Npts):
    price_plot_naive[k], var_plot_naive[k] = MC_bull_call_naive(S_plot[k],K1,K2,T,r,sigma,N)

# plot the 95% confidence interval as a shaded region (not noticeable as our Error is so small)
SEM = np.sqrt(var_plot_naive/N)
plt.fill_between(S_plot, price_plot_naive - 1.96*SEM, price_plot_naive + 1.96*SEM, 
                 alpha=0.5, color="darkorange")

# plot the MC Prices
plt.plot(S_plot, price_plot_naive,'-b',label = 'Naive MC')

#Overlay the BS Price
plt.plot(S_plot_BS, BSprice,marker = 'o', linestyle = '',color = 'r',alpha = 0.35, label = "Black-Scholes")




plt.xlabel("Spot price S at time = T", fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Naive MC Bull Call Spread Price for various Spot Prices")
plt.legend()
plt.show()



We will now repeat this process for the three types of variance reduction

# Monte Carlo With Antithetic Variance Reduction

As before we will investigate the variance plot and the max variance resulting from an initial 1000 simulations over stock prices of £10 to £200 in steps of 5 and determine our new sample size N to have a maximum absolute error of £0.01

In [ ]:
# Antithetic variance reduciton
# Resetting N back to our original 1000
#Plotting the variance for 1000 sims initally
N = 1000
S_plot = np.arange(10,205,5)
Npts = len(S_plot)
MC_price_ant = np.zeros(Npts)
MC_var_ant = np.zeros(Npts)


for k in range(Npts):
    MC_price_ant, MC_var_ant[k] = MC_bull_call_ant(S_plot[k],K1,K2,T,r,sigma,N)

plt.plot(S_plot,MC_var_ant, label = "Antithetic")

plt.xlabel("Spot price S")
plt.ylabel("Variance")
plt.title("Variance plot for Antithetic MC method ")
plt.legend()
plt.show()

#Saving variances for later plots
var_ant = MC_var_ant

In [ ]:
# Use max variance to determine sample sizes
Error = 0.01

Max_var = np.amax(MC_var_ant)
print("Max variance of naive is",'{0:.4g}'.format(Max_var))

# work out N needed for Error < 0.01
N = (1.96/Error)**2 * Max_var
print("For Error =", Error, "the needed sample size is", '{0:.3g}'.format(N))


In [ ]:
# Setting parameters 
S = 138 # Note how roughly a stock price of ~128 for antithetic method results in the maximum variance

# Our new N
N = int((1.96/Error)**2 * Max_var)

MC_price_ant, MC_var_ant = MC_bull_call_ant(S,K1,K2,T,r,sigma,N)

SE = np.sqrt(MC_var_ant/N)

print("Naive MC price is ", '£{0:.5g}'.format(MC_price_ant), "+/-", '£{0:.2g}'.format(1.96*SE) )

print("Standard Error = ",'£{0:.1g}'.format(SE))

#Saving Value for later
Nant = N

In [ ]:
#Now with our new sample size of N, we will price the spread using antithetic variance reduction

S_plot = np.arange(10,205,5)

# Our new N
N = int((1.96/Error)**2 * Max_var)

Npts = len(S_plot)
price_plot_ant = np.zeros( Npts )
var_plot_ant = np.zeros( Npts )

#Calculating prices with antithetic variance reduction 
for k in range(Npts):
    price_plot_ant[k], var_plot_ant[k] = MC_bull_call_ant(S_plot[k],K1,K2,T,r,sigma,N)



# plot the 95% confidence interval as a shaded region
SEM = np.sqrt(var_plot_ant/N)
plt.fill_between(S_plot, price_plot_ant - 1.96*SEM, price_plot_ant + 1.96*SEM, 
                 alpha=0.5, color="darkorange")

# plot the prices
plt.plot(S_plot, price_plot_ant,'-b',label = 'Antithetic')


#Overlay the BS Price
plt.plot(S_plot_BS, BSprice ,marker = 'o', linestyle = '',color = 'r',alpha = 0.35, label = "Black-Scholes")




plt.xlabel("Spot price S at time = T", fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("MC w/ Antithetic Variance Reduction Bull Call Spread Price")
plt.legend()
plt.show()

## Control Variates

The key aspect of using control variates to price options is to find another function $g(y)$ that is highly correlated with $f(y)$, but we know and can calculate the expectation of $g$. We do this becasue we want our sample mean of option prices to be an accurate estimation to the true mean of the payoff. In our case in order to find a function that was highly correlated with the payoff of a bull call spread, $f(S(T))$ we simply use the stock price $S(T)$

Having determined that $g(S(T))$ = $S(T)$ which is the geometric Brownian Motion, the mean and variance is known and are as follows:

<center> $$\bar g := \mathbb{E}[S(T)] = S(t)exp(rT)$$ </center>

<center> $$Var(g) := Var[S(T)] = (S(t)exp(rT))^2 (exp(\sigma^2T)-1) = \bar g^2(exp(\sigma^2T)-1)$$ </center>

With our control variate defined as 
<center> $$f_{c}(y) = f(y) -c(g(y) - \bar g)$$ </center>

Expectation of control variate:

<center> $$\mathbb{E}[f_{c}] = \mathbb{E}[f] - c(\mathbb{E}[g] - \bar g) = \mathbb{E}[f]$$ </center>

Hence variance of control variate:

<center> $$Var[f_{c}] = Var[f] - 2cCov[f,g] + c^2Var[g]$$ </center>

Then the optimal choice of c to minimise $var[f_c]$ is found by differentiating with respect to c and settting to zero.

Hence:

<center> $$ c = \frac{Cov[f,g]}{Var[g]}$$ </center>
    
Substituting this c into our original equation for the variance of $f_c$ gives the variance of the payoff as:
  
<center> $$Var[f_c] = Var[f](1-corr[f,g]^2)$$ </center>


Which shows us that if $g$ is highly correlated (or anti-correlated) to $f$ then the variance of $f_c$ will be small. The variance of $f_c$ will be zero in the case of perfect correlation or anti-correlation.

We will use this method and apply it to the case of the bull call spread.

## Monte Carlo With Variance Reduction Using Control Variates

In [ ]:
# Resetting N and investigating max variance and variance plot as before
N = 1000
S_plot = np.arange(10,205,5)
Npts = len(S_plot)
MC_price_con = np.zeros(Npts)
MC_var_con = np.zeros(Npts)

#Calculating prices with MC with control variates
for k in range(Npts):
    MC_price_con, MC_var_con[k] = MC_bull_call_con(S_plot[k],K1,K2,T,r,sigma,N)

plt.plot(S_plot,MC_var_con, label = "Control Variates")

plt.xlabel("Spot price S",fontsize="14")
plt.ylabel("Variance",fontsize="14")
plt.title("Variance plot for Control Variates MC method ")
plt.legend()
plt.show()

var_con = MC_var_con

In [ ]:
# Use max variance to determine sample sizes
Error = 0.01

Max_var = np.amax(MC_var_con)
print("Max variance of naive is", '{0:.4g}'.format(Max_var))

# work out N needed for Error < 0.01
N = (1.96/Error)**2 * Max_var
print("For Error =", Error, "the needed sample size is", '{0:.3g}'.format(N))

#Saving Value for later
Ncon = N

In [ ]:
#Test again
# Setting parameters
S = 130
N = int((1.96/Error)**2 * Max_var)



MC_price_con, var_plot_con = MC_bull_call_con(S,K1,K2,T,r,sigma,N)

SE = np.sqrt(var_plot_con/N)
print("Naive MC price is ", '£{0:.5g}'.format(MC_price_con), "+/-", '£{0:.2g}'.format(1.96*SE) )

print("Standard Error = ",'£{0:.1g}'.format(SE))

In [ ]:
#Plotting with our new sample size

N = int((1.96/Error)**2 * Max_var)

S_plot = np.arange(10,205,5)
Npts = len(S_plot)
price_plot_con = np.zeros( Npts )
var_plot_con = np.zeros( Npts )

startc = timeit.timeit()

for k in range(Npts):
    price_plot_con[k], var_plot_con[k] = MC_bull_call_con(S_plot[k],K1,K2,T,r,sigma,N)

endc = timeit.timeit()
timec = endc-startc
# plot the 95% confidence interval as a shaded region
SEM = np.sqrt(var_plot_con/N)
plt.fill_between(S_plot, price_plot_con - 1.96*SEM, price_plot_con + 1.96*SEM, 
                 alpha=0.5, color="darkorange")

# plot the MC Control Var Prices
plt.plot(S_plot, price_plot_con,'-b',label = 'Control Variate MC')

#Overlay the BS Price
plt.plot(S_plot_BS, BSprice,marker = 'o', linestyle = '',color = 'r',alpha = 0.35, label = "Black-Scholes")


plt.xlabel("Spot price S", fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Bull Call Spread Price for various Spot Prices Using Control Variates")
plt.legend()
plt.show()

## Importance Sampling

From the usual monte carlo framework, we view an integral as an expectation.

Where $Y$ is a uniformly distributed random variable.

Importance sampling utilises that instead of drawing samples from a uniform distribution, we can get a better estime for if we draw samples from a distribution that favours parts of the space where $f(Y)$ is large.
In essence we are chaging the probability measure over which we integrate.


In our case for using importance sampling for pricing options we deem the option price to be "important" when it is non-zero. As we are pricing a bull call spread whose pay-off is zero when it is not excercised which occurs when $S(T) < K_1$
With $K_1$ = £90, hence we will deem the range of $S(T)$ to be important when it is greater than or equal to £90

Writing the corresponding stock price and option price as follows:
$$S(T) = S(t)exp((r-\frac{1}{2}\sigma^2)T +\sigma\sqrt{T-t}\Phi^{-1}(Y))$$

$$V(S(T)) = \mathbb{E}[S(T)] = \int_{0}^{1}f(S(T,y))p_{1}(y)dy  = \int_{y_1}^{1}f(S(T,y))p_{1}(y)dy$$

Where $\Phi$ is the standard normal CDF, $y_1$ is defined implictly by 

$$S_{T}(y_{1}) = K $$

Now we want to condsider $p_2(y)$, the pdf for the uniform distribution on $[y_1,1]$ instead of $p_1$, the the standard uniform distribution.

$$ p_{2}(y) \begin{cases}
      \frac{1}{1-y_1}, & y_1\leq y \leq 1 
      0, & 0 \leq y \leq y_1
    \end{cases}$$
    
Then

$$ V = \int_{y_1}^{1} f(S_T(y))\frac{p_1(y)}{p_2(y)}p_2(y)dy $$

  $$  = (1-y_1)\int_{y_1}^{1} f(S_T(y))p_2(y)dy$$
  
By restricting the integral to $[y_1,1]$ we have removed the portion of the interval where $p_2$ would be zero. The factor $(1-y_1)$ is needed to account for the fact that we have changed probability measures from the uniform on $[0,1]$ to the uniform on $[y_1,1]$

We will use this general method to price the bull call spread.


### Monte Carlo With Variance Reduction Using Importance Sampling

In [ ]:
#Reset N

N = 1000
S_plot = np.arange(10,205,5)
Npts = len(S_plot)
MC_price_imp = np.zeros(Npts)
MC_var_imp = np.zeros(Npts)


for k in range(Npts):
    MC_price_imp, MC_var_imp[k] = MC_bull_call_imp(S_plot[k],K1,K2,T,r,sigma,N)

plt.plot(S_plot,MC_var_imp, label = "Importance Sampling")

plt.xlabel("Spot price S",fontsize="14")
plt.ylabel("Variance",fontsize="14")
plt.title("Variance plot for Importance Sampling MC method ")
plt.legend()
plt.show()
var_imp = MC_var_imp

In [ ]:
# Use max variance to determine sample sizes
Error = 0.01

Max_var = np.amax(MC_var_imp)
print("Max variance of importance sampling MC is", '{0:.5g}'.format(Max_var),"\n")

# work out N needed for Error < 0.01
N = (1.96/Error)**2 * Max_var
print("For Error =", Error, "the needed sample size is", '{0:.3g}'.format(N))


In [ ]:
#Test again
# Setting parameters
S = 128
N = int((1.96/Error)**2 * Max_var)

MC_price_imp, MC_price_imp= MC_bull_call_imp(S,K1,K2,T,r,sigma,N)

SE = np.sqrt(MC_price_imp/N)
print("Naive MC price is £", '{0:.5g}'.format(MC_price_imp), "+/-", '{0:.2g}'.format(1.96*SE) )

print("Standard Error = ",'£{0:.1g}'.format(SE))

#Saving Value for later
Nimp = N

In [ ]:
#Now with our new sample size of N, we will price the spread using antithetic variance reduction

S_plot = np.arange(10,205,5)
Npts = len(S_plot)
price_plot_imp = np.zeros( Npts )
var_plot_imp = np.zeros( Npts )

starti = timeit.timeit()
for k in range(Npts):
    price_plot_imp[k], var_plot_imp[k] = MC_bull_call_imp(S_plot[k],K1,K2,T,r,sigma,N)

endi = timeit.timeit()
timei = endi-starti

    
# plot the 95% confidence interval as a shaded region
SEM = np.sqrt(var_plot_imp/N)
plt.fill_between(S_plot, price_plot_imp - 1.96*SEM, price_plot_imp + 1.96*SEM, 
                 alpha=0.5, color="darkorange")

# plot the mean
plt.plot(S_plot, price_plot_imp,'-b', label = 'Importance Sampling MC')


#Overlay the BS Price
plt.plot(S_plot_BS, BSprice ,marker = 'o', linestyle = '',color = 'r',alpha = 0.35, label = "Black-Scholes")

plt.xlabel("Spot price S", fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Bull Call Spread Price for various Spot Prices Using Importance Sampling")
plt.legend()
plt.show()

The following plot displays the variances of the four different methods used to calculate the price of the bull call spread according to the different stock prices. As we can see the three variance reduction techniques significantly reduce the variance of our prices.

In [ ]:
plt.figure(figsize = (7,7))

plt.plot(S_plot,var_naive, label = "Naive")
plt.plot(S_plot,var_ant, label = "Antithetic")
plt.plot(S_plot,var_imp, label = "Importance Sampling")
plt.plot(S_plot,var_con, label = "Control Variates")



plt.xlabel("Spot price S",fontsize="14")
plt.ylabel("Variance",fontsize="14")
plt.title("Variance plot for all MC methods")
plt.legend()
plt.show()

### Variance Reduction Discussion

Looking the graph above before calculating the sample sizes needed for the 4 methods to have an absolute error of £0.01 would suggest that the Antithetic variance reduction would require less iterations thus being more efficient than the other three methods.

Below presents the sample sizes needed for our required accuracy of £0.01 for the four methods and their corresponding N iterations to price the bull call spread for the same range of stock prices.

|| Naive | Antithetic | Control Variates | Importance Sampling |
| :-:| :-: | :-: |:-:|:-: |
|Required Sample Size |6.08 x 10^6| 1.21 x 10^6 |2.02 x 10^6| 1.96x 10^6 |


----------
## Delta

To compute the delta we will use a combination of path recycling and antithetic variance reduction. We use path recycling as opposed to standard finite difference methods because the variance of the standard finite differencing methods can be large, and can cause problems if the change in the parameter theta tends to zero.

We consider the centered finite difference approach as follows :

$$ \frac{\partial V}{\partial \theta} (\theta_j) = \frac{V_{j+1} - V_{j-1}}{2\Delta\theta} + \mathcal{O}(\Delta\theta^2) $$

Where $\Delta\theta$ = $\theta_{j+1} - \theta{j}$ is constant for all values.

Hence we use the same sample $X_1,.....,X_i,....$ to compute both $ V_{j+1}$ and $ V_{j-1}$ Their correlation will then no longer be zero this is the key idea in path recyling.

So we have:

$$ Var\left[\frac{V_{j+1} - V_{j-1}}{2\Delta\theta}\right] \approx Var\left[\frac{dV}{d\theta}\theta_j \right] $$

Where the variance on the right-hand-side is the variance in a sample mean of the derivative we are trying to approximate. We will then combine this pathrecycling method with antithetic variance reduction techniques which we are familiar with.

We could also compute the delta using pathwise derivatives with antithetic variance reduction or using likelihood ratio methods but these methods can get complicated for more exotic and complicated options where the formula for the derivative isnt easily calculated analytically. But one useful feature of using likelihood ratio methods is that it can be used in cases where the option payoff is discontinuous.



In [ ]:
#Reset N
N = 1000

S_plot = np.arange(10,205,5)
Npts = len(S_plot)
BSprice = np.zeros(Npts)
MC_delta_ant_path = np.zeros(Npts)
MC_var_delta_ant_path = np.zeros(Npts)
for k in range(Npts):
    BSprice[k] = BS_bull_call_delta(S_plot[k],K1,K2,T,r,sigma)
    MC_delta_ant_path[k], MC_var_delta_ant_path[k] = MC_bull_call_delta_ant_path(S_plot[k],K1,K2,T,r,sigma,N)

plt.plot(S_plot,MC_delta_ant_path, label = "MC Path Delta")

plt.xlabel("Spot price S",fontsize="14")
plt.ylabel("Variance",fontsize="14")

plt.plot(S_plot_BS, BSprice,marker = 'o', linestyle = '',color = 'r',label = "BS",alpha = 0.4)

plt.xlabel("Spot price S", fontsize="14")
plt.ylabel("Bull Call Spread Delta", fontsize="14")
plt.title("Plot of Bull Call Spread Deltas")
plt.legend()
plt.show()


## Exploratoratory Analysis

First we will look at how the the option price vary with a variety of maturity times T, using Antithetic Variance reduction as our MC method of choice.

In [ ]:
#Now with our new sample size of N, we will price the spread using antithetic variance reduction
#Setting our maturity times from 0.5 years to 3 years
T_s = [0.5,1.0,1.5,2.5,3.5,4,4.5,5,10]

S_plot = np.arange(0,205,1)
Ncols = len(T_s)
q = np.arange(0,Ncols,1)
Npts = len(S_plot)
#arrays to store contract prices for each maturity time
price_plot_ant = np.zeros([Npts,Ncols])
var_plot_ant = np.zeros([Npts,Ncols])

for i in q:
    T = T_s[i]
    for k in range(Npts):
        price_plot_ant[k,i], var_plot_ant[k,i] = MC_bull_call_ant(S_plot[k],K1,K2,T,r,sigma,N)
    
    plt.plot(S_plot,price_plot_ant[:,i], label = 'T =' + str(T) + '')



plt.xlabel("Spot price S" ,fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Bull Call Spread Price for different maturity times T")
plt.legend()
plt.show()


Surface Plot for the same times to maturity T for illustration.

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
x = np.arange(0,205,1)
y = [0.5,1.0,1.5,2.5,3.5,4,4.5,5,10]
#create mesh grid of times and stock prices
X, Y = np.meshgrid(y,x)
zs = price_plot_ant
z = ax.plot_surface(X,Y,zs, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)


ax.view_init(40 ,-30 )
ax.set_xlabel("Time To Expiry")
ax.set_ylabel("Spot Price")
ax.set_zlabel("Value of Contract")

plt.title("Surface Plot of Bull Call Spread for various Expiry Times")
plt.show()

### We will repeat the above for the delta of the option


In [ ]:
#Now with our new sample size of N, we will price the spread using antithetic variance reduction
#Setting our maturity times from 0.5 years to 3 years
T_s = [0.5,1.0,1.5,2.5,3.5,4,4.5,5,10]

S_plot = np.arange(10,205,5)
Ncols = len(T_s)
q = np.arange(0,Ncols,1)
Npts = len(S_plot)
price_plot_d_p = np.zeros([Npts,Ncols])
var_plot_d_p = np.zeros([Npts,Ncols])

for i in q:
    T = T_s[i]
    for k in range(Npts):
        price_plot_d_p[k,i], var_plot_d_p[k,i] = MC_bull_call_delta_ant_path(S_plot[k],K1,K2,T,r,sigma,N)
    
    plt.plot(S_plot,price_plot_d_p[:,i], label = 'T =' + str(T) + '')



plt.xlabel("Spot price S" ,fontsize="14")
plt.ylabel("Bull Call Spread value", fontsize="14")
plt.title("Bull Call Spread Delta for different maturity times T")
plt.legend()
plt.show()



In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
x = np.arange(10,205,5)
y = [0.5,1.0,1.5,2.5,3.5,4,4.5,5,10]
X, Y = np.meshgrid(y,x)
zs = price_plot_d_p
z = ax.plot_surface(X,Y,zs, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)


ax.view_init(20 ,-30)
ax.set_xlabel("Time To Expiry")
ax.set_ylabel("Spot Price")
ax.set_zlabel("Delta")

plt.title("Surface Plot of Bull Call Spread Delta for various Expiry Times")
plt.show()

## Discussion

From our analysis of our different monte carlo methods, using antithetic variance reduction performed the best in terms of needing the smallest sample size of all the methods to have a maxiumum absolute error of 0.01 hence the variance reduction performed the best here. With the smaller sample size this method will also be faster than the other methods which can be important in a high frequency trading setting where speed and accuracy is essential. 

In our exploratory analysis we see how that the longer the maturity time, the lower the delta of the contract for the same spot prices, which makes sense as the change in the option value will fluctuate less if there is a longer time to maturity. However when the spot price is far lower than the first strike price K1 we notice how the option with maturity time 10 has a higher price than that of contracts with a smaller time to maturity. This is because there is more time for the spot price to move in a direction that would bring the option in the money for us, but as the spot price gets closer to K1 the shorter maturity time contracts start to become more valuable. (This may not always be true however depending on how large sigma (volatility) is